In [66]:
import torch
import numpy as np
import pickle

In [ ]:
dataset = "GB1" # "GB1", "PhoQ"

In [ ]:
f = open("../de/data/boes/" + dataset + "/BOSS_v0.2.0_24_budget/variant_prediction.pkl", "rb")
#x1, x2, y, y_variance, y_variants = pickle.load(f)
y, y_variance, y_variants = pickle.load(f)

In [69]:
f = torch.load("./../../data/"+dataset+"/esm-1b_fitness_norm.pt") #df["Fitness"].values
v = torch.load("./../../data/"+dataset+"/esm-1b_variants.pt") #df["Variants"].values
d = dict(map(lambda i,j : (i,j), v, f))

embedding = torch.load("./../../data/"+dataset+"/esm-1b_embedding_complete.pt")
variants = torch.load("./../../data/"+dataset+"/esm-1b_variants_complete.pt")
fitness = [d.get(v, torch.tensor(0., dtype=torch.float64)) for v in variants]

C:\Users\Matouš\AppData\Local\Temp\ipykernel_16892\2233195592.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  f = torch.load("./../../data/"+dataset+"/esm-1b_fitness_nor

Sort predicted and ground-truth variants to match

In [70]:
data = [(embedding[i], variants[i], fitness[i]) for i in range(len(embedding))]

new_indexes = [variants.index(v) for v in y_variants]
data = [data[i] for i in new_indexes]

embedding = [d[0] for d in data]
variants = [d[1] for d in data]
fitness = [d[2] for d in data]

In [71]:
variants == y_variants

True

In [72]:
from sklearn.metrics import ndcg_score, dcg_score
from scipy.stats import pearsonr, spearmanr

In [73]:
pr = pearsonr(fitness, y)
pr

PearsonRResult(statistic=0.17223682901353043, pvalue=0.0)

In [74]:
sr = spearmanr(fitness, y)
sr

SignificanceResult(statistic=0.22908209379482794, pvalue=0.0)

In [75]:
dcg = dcg_score([fitness], [y])
dcg

52.71310972456477

In [76]:
ndcg = ndcg_score([fitness], [y])
ndcg

0.7382899936905117